In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from __future__ import print_function
import keras
from keras.utils import to_categorical
import os
from keras.preprocessing.image import ImageDataGenerator, load_img

Using TensorFlow backend.


In [3]:
from keras.applications import VGG16

vgg_conv = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(224, 224, 3))

58892288/58889256 [==============================] - 1s 0us/step


In [5]:
from keras.applications import inception_v3,resnet50,mobilenet
inception_model =inception_v3.InceptionV3(weights = 'imagenet' ,
                  include_top=False,
                  input_shape=(224, 224, 3))
resnet_model = resnet50.ResNet50(weights = 'imagenet' ,
                  include_top=False,
                  input_shape=(224, 224, 3))
mobilenet_model = mobilenet.MobileNet(weights = 'imagenet' ,
                  include_top=False,
                  input_shape=(224, 224, 3))

17227776/17225924 [==============================] - 1s 0us/step


In [6]:
vgg_conv.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [7]:
vgg_conv.output

<tf.Tensor 'block5_pool/MaxPool:0' shape=(?, 7, 7, 512) dtype=float32>

In [8]:
train_dir = './clean-dataset/train'
validation_dir = './clean-dataset/validation'

nTrain = 600
nVal = 150

In [9]:
datagen = ImageDataGenerator(rescale=1./255)
batch_size = 20

train_features = np.zeros(shape=(nTrain, 7, 7, 512))
train_labels = np.zeros(shape=(nTrain,3))

In [10]:
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True)

Found 600 images belonging to 3 classes.


In [11]:
i = 0
for inputs_batch, labels_batch in train_generator:
    features_batch = vgg_conv.predict(inputs_batch)
    train_features[i * batch_size : (i + 1) * batch_size] = features_batch
    train_labels[i * batch_size : (i + 1) * batch_size] = labels_batch
    i += 1
    if i * batch_size >= nTrain:
        break
        
train_features = np.reshape(train_features, (nTrain, 7 * 7 * 512))

In [12]:
train_features[0]

array([0.        , 0.        , 0.        , ..., 0.        , 0.85347462,
       0.        ])

In [13]:
train_features.shape

(600, 25088)

In [14]:
validation_features = np.zeros(shape=(nVal, 7, 7, 512))
validation_labels = np.zeros(shape=(nVal,3))

validation_generator = datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

i = 0
for inputs_batch, labels_batch in validation_generator:
    features_batch = vgg_conv.predict(inputs_batch)
    validation_features[i * batch_size : (i + 1) * batch_size] = features_batch
    validation_labels[i * batch_size : (i + 1) * batch_size] = labels_batch
    i += 1
    if i * batch_size >= nVal:
        break

validation_features = np.reshape(validation_features, (nVal, 7 * 7 * 512))

Found 150 images belonging to 3 classes.


In [15]:
validation_features[0]

array([0.08734369, 0.        , 0.        , ..., 0.        , 0.61830747,
       0.        ])

### Create your own model and train the network

In [16]:
from keras import models
from keras import layers
from keras import optimizers

model = models.Sequential()
model.add(layers.Dense(512, activation='relu', input_dim=7 * 7 * 512))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(3, activation='softmax'))

model.compile(optimizer=optimizers.RMSprop(lr=2e-4),
              loss='categorical_crossentropy',
              metrics=['acc'])

history = model.fit(train_features,
                    train_labels,
                    epochs=20,
                    batch_size=batch_size,
                    validation_data=(validation_features,validation_labels))

Train on 600 samples, validate on 150 samples
Epoch 1/20
600/600 [==============================] - 8s 14ms/step - loss: 1.7602 - acc: 0.5683 - val_loss: 0.4104 - val_acc: 0.8600
Epoch 2/20
600/600 [==============================] - 5s 9ms/step - loss: 0.5133 - acc: 0.8083 - val_loss: 1.3224 - val_acc: 0.5733
Epoch 3/20
600/600 [==============================] - 5s 9ms/step - loss: 0.3016 - acc: 0.8883 - val_loss: 0.3379 - val_acc: 0.8733
Epoch 4/20
600/600 [==============================] - 5s 9ms/step - loss: 0.1581 - acc: 0.9300 - val_loss: 0.3214 - val_acc: 0.8800
Epoch 5/20
600/600 [==============================] - 5s 9ms/step - loss: 0.1857 - acc: 0.9350 - val_loss: 0.8339 - val_acc: 0.7733
Epoch 6/20
600/600 [==============================] - 5s 9ms/step - loss: 0.0804 - acc: 0.9767 - val_loss: 0.3240 - val_acc: 0.9133
Epoch 7/20
600/600 [==============================] - 5s 9ms/step - loss: 0.0518 - acc: 0.9817 - val_loss: 0.3513 - val_acc: 0.8933
Epoch 8/20
600/600 [=========

In [17]:
fnames = validation_generator.filenames

ground_truth = validation_generator.classes

label2index = validation_generator.class_indices

# Getting the mapping from class index to class label
idx2label = dict((v,k) for k,v in label2index.iteritems())

AttributeError: 'dict' object has no attribute 'iteritems'

In [ ]:
idx2label[0]

In [ ]:
predictions = model.predict_classes(validation_features)
prob = model.predict(validation_features)

In [ ]:
errors = np.where(predictions != ground_truth)[0]
print("No of errors = {}/{}".format(len(errors),nVal))

### Let us see which images were predicted wrongly

In [ ]:
for i in range(len(errors)):
    pred_class = np.argmax(prob[errors[i]])
    pred_label = idx2label[pred_class]
    
    print('Original label:{}, Prediction :{}, confidence : {:.3f}'.format(
        fnames[errors[i]].split('/')[0],
        pred_label,
        prob[errors[i]][pred_class]))
    
    original = load_img('{}/{}'.format(validation_dir,fnames[errors[i]]))
    plt.imshow(original)
    plt.show()